# 🦌 ELK App 🚀

This is a high-level widget for interacting with ELK Transformers and ELK Diagrams

In [2]:
import json
import pathlib

import importnb
import ipywidgets
import networkx
import traitlets

import ipyelk
import ipyelk.nx
# import ipyelk.tools
# import ipyelk.tools.tools

In [3]:
with importnb.Notebook():
    from __02_Transformer import a_hierarchical_elk_example

KeyboardInterrupt: 

In [ ]:
ipyelk.Diagram(
    data=xelk,
    transformer=Mine(),
    
)


In [ ]:
ipyelk.Diagram(nx.Graph())


class ElkGraph():
    relationships:
    hierarchy:

In [ ]:
ipyelk.Diagram({})

In [ ]:
def an_elk_app_example(elk_xelk=None):
    elk, xelk = elk_xelk or a_hierarchical_elk_example()
    app = ipyelk.Elk(transformer=xelk, layout=dict(display="flex", flex="1"))
    toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    box = ipywidgets.VBox(
        [
            ipywidgets.HBox(
                [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
            ),
            app,
        ],
        layout=dict(height="100%"),
    )
    return app, box, xelk, elk

In [ ]:
if __name__ == "__main__":
    app, box, xelk, elk = an_elk_app_example()
    display(box)

#### Command Buttons can be customized to tailor the look and feel

In [ ]:
def an_elk_app_with_a_custom_tool_button():
    app, box, xelk, elk = an_elk_app_example()
    fit = app.toolbar.commands[0]
    fit.icon = "arrows-alt"
    fit.tooltip = "Fit Diagram"
    fit.layout.width = "5em"
    fit.description = ""
    return app, box, xelk, elk

In [ ]:
if __name__ == "__main__":
    display(an_elk_app_with_a_custom_tool_button()[1])

#### Demonstrating adding close button

Adding a callable to an `Elk` Application's Toolbar `on_close` will result in a close
button appearing in the toolbar which will fire that function when pressed.

In [ ]:
def an_elk_app_with_custom_close():
    app, box, xelk, elk = an_elk_app_example()

    out = ipywidgets.Output()
    with out:
        print("  ┌ Press the close button!\n" "  │\n" "  │\n" "  │\n" "◀─╯")

    def simulating_close():
        with out:
            print("─▸Toolbar close button pressed!")

    app.toolbar.on_close = simulating_close
    return app, ipywidgets.HBox([box, out]), xelk, elk

In [ ]:
if __name__ == "__main__":
    display(an_elk_app_with_custom_close()[1])

## Custom Styling For Collaping Hierarchical Example

The style dictionary takes the first key and adds a parent selector based on the current
`StyleWidget`'s id. This helps to namespace the css selectors but also means that the
leading space in the style dictionary keys is intentional and is a descendant selector
from the root.

The `.slack-port` and `.slack-edge` are custom css classes applied by the XELK
transformer when collapsing a node causes either a source or destination of a tunneling
edges to disapear requiring the introduction of a new edge and port at a higher level.

In [ ]:
def a_styled_elk_app_example():
    app, box, xelk, elk = an_elk_app_example()
    app.style = {
        " rect": {"opacity": ".75", "transition": "all 0.2s"},
        " .slack-port>rect": {
            "fill": "red",
            "opacity": ".25",
            "transition": "all 0.2s",
        },
        " .slack-edge>path": {
            "stroke": "red",
            "opacity": ".25",
            "stroke-dasharray": "4",
            "transition": "all 0.2s",
            "stroke-width": "2px",
        },
        " .slack-edge>path.edge.arrow": {"fill": "red", "opacity": ".25"},
    }
    return app, box, xelk, elk

In [ ]:
if __name__ == "__main__":
    display(a_styled_elk_app_example()[1])

### Custom `ElkNode` css classes

Custom css classes can be passed to `ElkNode`, `ElkPort`, and `ElkEdge`. The `XELK`
transformer has a nested dictionary where the keys are nodes in the source networkx
graph and the values are another dictionary with Elk types as keys and values are sets
of strings. This set gets applied to the Elk types that are generated for the given
networkx node.

In [ ]:
from ipyelk.diagram.elk_model import ElkEdge, ElkLabel, ElkNode, ElkRoot


def a_more_stylish_elk_app_example():
    app, box, xelk, elk = a_styled_elk_app_example()
    app.transformer.css_classes = {
        "n1": {
            ElkNode: set(["example-custom-class"]),
            ElkLabel: set(["example-custom-text"]),
            #         shape:"shoe",
        },
        ElkRoot: {
            ElkEdge: set(
                ["example-styled-edge"]
            ),  # colors edges under the root ElkJson Node
        },
    }

    elktyped_css = {
        " .example-custom-class>rect.elknode": {"fill": "grey", "opacity": ".25"},
        " .example-styled-edge>.elkedge": {"stroke": "green"},
        " text.example-custom-text.elklabel": {"fill": "red"},
    }
    app.style = {**app.style, **elktyped_css}
    app.refresh()
    return app, box, xelk, elk

In [ ]:
if __name__ == "__main__":
    display(a_more_stylish_elk_app_example()[1])

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).